In [15]:
#import packages
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from datetime import date 
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import statsmodels.api as sm
import duckdb, sqlalchemy

%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# Data Cleaning

We obtained our data through 3 data requests to weather.gov. First, we requested Ithaca's data, and then we decided to expand out analysis to include locations north, east, south, and west of Ithaca. Our second data pull included the following cities: Watertown (North), Bloomsburg (South), Cobleskill (East), and Avoca (West). Avoca did not contain any temperature data, so we decided to request data for Erie as a replacement. The cell below, reads in the csv files obtained from weather.gov.

In [20]:
#read in csvs
ithaca = pd.read_csv("Ithaca.csv")
others = pd.read_csv("AdjacentCities.csv")
west = pd.read_csv("Erie.csv")
#print(ithaca.head())
print(west['NAME'].unique())
others = others.dropna(axis=0,subset=['STATION'])
print(others['NAME'].unique())

C:\Users\Margaret Quirk\AppData\Local\Temp\ipykernel_25496\2358469234.py:2: DtypeWarning: Columns (9,15,17,19,21,27,29,31,33,35,37,39,41,47) have mixed types. Specify dtype option on import or set low_memory=False.
  ithaca = pd.read_csv("Ithaca.csv")
C:\Users\Margaret Quirk\AppData\Local\Temp\ipykernel_25496\2358469234.py:3: DtypeWarning: Columns (27,29,31,33,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  others = pd.read_csv("AdjacentCities.csv")


['NORTH EAST 1.2 WNW, PA US' 'ERIE 4.4 ESE, PA US' 'ERIE 5.7 SSE, PA US'
 'ERIE 5.6 SW, PA US' 'NORTH EAST 2.0 S, PA US' 'ERIE 5.3 E, PA US'
 'ERIE INTERNATIONAL AIRPORT, PA US']
['BLACK RIVER, NY US' 'BLOOMSBURG UNIVERSITY, PA US'
 'COBLESKILL 2 ESE, NY US' 'COBLESKILL 4.2 NNE, NY US'
 'COBLESKILL 5.7 W, NY US' 'ESPY 0.8 S, PA US' 'WATERTOWN 0.2 SSE, NY US'
 'WATERTOWN 1.0 ESE, NY US' 'WATERTOWN 1.3 WNW, NY US'
 'WATERTOWN 7.0 N, NY US' 'WATERTOWN, NY US']


C:\Users\Margaret Quirk\AppData\Local\Temp\ipykernel_25496\2358469234.py:4: DtypeWarning: Columns (7,9,11,13,15,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81) have mixed types. Specify dtype option on import or set low_memory=False.
  west = pd.read_csv("Erie.csv")


Since certain locations contained multiple weather stations, we created the column Location to group the entries as shown in the cell below.

In [3]:
#Add column to indicate location (relative to ithaca)
ithaca["Location"] = "Central"
west['Location'] = "West"
#print(others['NAME'].unique())
locations = []
for i in others['NAME']:
    if "WATERTOWN" in i:
        locations.append('North')
    elif "BLACK RIVER" in i:
        locations.append("North")
    elif "ESPY" in i:
        locations.append("South")
    elif "BLOOMSBURG" in i:
        locations.append("South")
    elif "COBLESKILL" in i:
        locations.append("East")
    else:
        locations.append('N/A')
others['Location'] = locations
print(others['Location'].unique())

['North' 'South' 'East']


Certain cities had more data attributes available than others, so to maintain cohesion, the cell below identifies common columns between the three data sets. We then dropped any uncommon columns and concatenated the three datasets to form one dataframe containing all relevant data.

In [4]:
#Check all columns match before concatenation
columns_to_keep = []

for i in ithaca.columns.tolist():
    if (i in others.columns.tolist()) & (i in west.columns.tolist()) :
        columns_to_keep.append(i)
ithaca_good = ithaca[columns_to_keep]
others_good = others[columns_to_keep]
west_good = west[columns_to_keep]
        
if (ithaca_good.columns.tolist() == others_good.columns.tolist()) & (ithaca_good.columns.tolist() == west_good.columns.tolist()):
    print("Columns match: proceed")
    final_df = pd.concat([ithaca_good,others_good,west_good])
else:
    print(ithaca.columns)
    print(others.columns)
    
#print(final_df)

Columns match: proceed


In the cell below, we checked to make sure that none of the columns contained only null values. All columns contained at least one relevant datapoint, but if this was not the case, we also included code that would drop an entirely null column.

In [5]:
#Check for null values in columns
print(final_df.columns)
#If a column has only null values, drop the column
null_cols = []
for c in final_df.columns:
    if final_df[c].isnull().all():
        null_cols.append(c)
if len(null_cols) == 0:
    print("No null columns")
else:
    print('Null columns:', null_cols)
    final_df = final_df.drop(null_cols)

Index(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'DAPR',
       'DAPR_ATTRIBUTES', 'MDPR', 'MDPR_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES',
       'SNOW', 'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES', 'TMAX',
       'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES', 'WESD', 'WESD_ATTRIBUTES',
       'WESF', 'WESF_ATTRIBUTES', 'WT05', 'WT05_ATTRIBUTES', 'Location'],
      dtype='object')
No null columns


The attributes columns in the weather.gov data contains a string that contains multiple attribute indicators concatenated together. "Trace" is an attribute that we believe will be relevant to our analysis and is represented by a "T" in the attribute columns. We used this to create new binary columns that indicate whether or not there was a trace of precipitation or snow on a given day.

In [6]:
#Create binary columns to indicate if there was trace of precipitation for snow and rain
precip_binary = []
snow_binary = []
for p in final_df['PRCP_ATTRIBUTES']:
    #print(type(p))
    if (type(p) == str):
        if 'T' in p:
            precip_binary.append(1)
        else:
            precip_binary.append(0)
    else:
        precip_binary.append(0)
        
for s in final_df['SNOW_ATTRIBUTES']:
    if (type(s) == str):
        if 'T' in s:
            snow_binary.append(1)
        else:
            snow_binary.append(0)
    else:
        snow_binary.append(0)
    
final_df['PrecipTrace'] = precip_binary
final_df['SnowTrace'] = snow_binary

With the addition of the binary columns, we no longer need the initial attributes columns and drop them from the dataframe below.

In [7]:
#Drop attributes columns now that we have created the binary columns
final_df = final_df.drop(columns = ['PRCP_ATTRIBUTES', 'SNOW_ATTRIBUTES'])

In addition, we converted the data column to datetime.

In [8]:
#Convert date column to datetime
final_df['DATE'] = pd.to_datetime(final_df['DATE'], format = '%m/%d/%Y')
#print(final_df['DATE'])

We also created a column that indicates the season. We categorized seasons as:

Winter: December-February

Spring: March-May

Summer: June-August

Fall: September-November

In [10]:
season_list = []
for i in final_df['DATE']:
    if (i.month == 12) or (i.month == 1) or (i.month == 2):
        season_list.append('Winter')
    elif (i.month == 3) or (i.month == 4) or (i.month == 5):
        season_list.append('Spring')
    elif (i.month == 6) or (i.month == 7) or (i.month == 8):
        season_list.append('Summer')
    elif (i.month == 9) or (i.month == 10) or (i.month == 11):
        season_list.append('Fall')
    else:
        print("ERROR")
final_df['Season'] = season_list

In some cases, there were multiple stations within the same location category that recorded data. To account for this, we created an aggregated dataframe that groups by location and date and takes the average of the temperature, location, elevation, and precipitation data. This aggregated dataframe takes the maximum of the binary columns to indicate if there was a trace anywhere within the location. We wanted to maintain the binary property of the columns in order to perform logistic regression in the final phase of the project.

In [11]:
#Aggregate dataframe to account for locations with multiple stations taking recordings on the same day
%sql agg_df << select DATE, Location, avg(LATITUDE) as Latitude, avg(LONGITUDE) as Longitude, avg(ELEVATION) as Elevation, AVG(TMAX) as MaxTemp, AVG(TMIN) as MinTemp,AVG(PRCP) as Precipitation, AVG(SNOW) as Snowfall, MAX(PrecipTrace) as PrecipTrace, MAX(SnowTrace) as SnowTrace, Season from final_df group by Location, DATE, Season order by DATE
agg_df['AverageTemp'] = 0.5*agg_df['MaxTemp'] + 0.5*agg_df['MinTemp']

agg_df['Month_Grouping'] = agg_df['DATE'].dt.to_period('M')
agg_df['Month_Grouping'] = agg_df['Month_Grouping'].astype('datetime64[M]')
#print(agg_df[['DATE','Month Grouping']])

Returning data to local variable agg_df


We also split the aggregated dataframe into smaller dataframes based on location.

In [13]:
#Create 5 separate data frames based on location for individual analyses when needed
central = agg_df[agg_df['Location']=="Central"]
north = agg_df[agg_df['Location']=="North"]
south = agg_df[agg_df['Location']=="South"]
east = agg_df[agg_df['Location']=="East"]
west = agg_df[agg_df['Location']=="West"]


In [14]:
#Export agg_df
agg_df.to_csv('agg_df.csv')